In [ ]:
pip install pandas

In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_openai import ChatOpenAI


In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
KEY=os.getenv("OPENAI_API_KEY")


In [22]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [7]:

llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.3)

In [8]:
llm


ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000002579ABE4EC0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002579ABE5940>, root_client=<openai.OpenAI object at 0x000002579977DFD0>, root_async_client=<openai.AsyncOpenAI object at 0x000002579ABE56A0>, temperature=0.3, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [9]:
%pip install langchain-community
%pip install PyPDF2





Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.callbacks import get_openai_callback
from PyPDF2 import PdfReader
import os
from langchain_core.output_parsers import StrOutputParser

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} briefly."
)

chain = prompt | llm | StrOutputParser()

In [12]:
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000002579AF49310>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002579AF496D0>, root_client=<openai.OpenAI object at 0x000002579AF49090>, root_async_client=<openai.AsyncOpenAI object at 0x000002579AF49450>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [14]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [41]:
TEMPLATE="""

Text:
{text}

You are an expert MCQ maker.

STRICT INSTRUCTIONS:
- Return ONLY valid JSON
- Do NOT include explanations
- Do NOT include markdown
- Do NOT include text outside JSON

The JSON format MUST be exactly:

{
  "quiz": [
    {
      "question": "string",
      "options": {
        "A": "string",
        "B": "string",
        "C": "string",
        "D": "string"
      },
      "answer": "A"
    }
  ]
}

Create {number} questions for {subject} students in a {tone} tone.
"""


In [42]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template=TEMPLATE
    )

In [17]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [20]:
review_chain= quiz_evaluation_prompt| llm | StrOutputParser()

In [21]:
generate_evaluate_chain = quiz_chain | review_chain


In [22]:
file_path=r"C:\Users\Disha shetty\MCQgenerator\data.txt"

In [23]:
file_path

'C:\\Users\\Disha shetty\\MCQgenerator\\data.txt'

In [24]:
with open(file_path, 'r') as file:
    TEXT = file.read()


In [25]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[3][4]

From a theoretical viewpoint, probably approximately corr

In [26]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
NUMBER=5 
SUBJECT="MACHINE LEARNING"
TONE="simple"

In [28]:
from langchain_community.callbacks import get_openai_callback


In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# Quiz prompt
quiz_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template="""
Text:
{text}

You are an expert MCQ maker.

STRICT INSTRUCTIONS:
- Return ONLY valid JSON
- Do NOT include explanations
- Do NOT include markdown
- Do NOT include text outside JSON

The JSON format MUST be exactly:

{{
  "quiz": [
    {{
      "question": "string",
      "options": {{
        "A": "string",
        "B": "string",
        "C": "string",
        "D": "string"
      }},
      "answer": "A"
    }}
  ]
}}

Create {number} MCQs for {subject} students in a {tone} tone.
"""
)


# Quiz chain
quiz_chain = quiz_prompt | llm | StrOutputParser()

# Review prompt
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["quiz"],
    template="Review the following quiz and suggest improvements:\n{quiz}"
)

# Review chain
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

# Adapter (VERY IMPORTANT)
adapt_quiz_output = RunnableLambda(lambda quiz: {"quiz": quiz})

# FINAL chain (this is what was missing)
generate_evaluate_chain = quiz_chain | adapt_quiz_output | review_chain


In [47]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE
        }
    )

print(response)
print(cb)





The quiz you've provided is a good starting point for assessing knowledge of machine learning concepts. However, there are several areas where improvements can be made to enhance clarity, engagement, and educational value. Here are some suggestions:

### 1. **Question Variety and Depth**
- **Increase Complexity**: Some questions are quite basic. Consider adding questions that require a deeper understanding or application of concepts.
- **Include Real-World Scenarios**: Incorporate questions that present real-world scenarios where machine learning is applied, asking the quiz taker to identify the correct approach or method.

### 2. **Answer Choices**
- **Distractor Quality**: Ensure that distractor options (incorrect answers) are plausible and relevant. This will make the quiz more challenging and educational.
- **More Options**: Consider providing more than four options for each question. This can increase the difficulty and require more critical thinking.

### 3. **Clarity and Precisi

In [49]:
print(type(response))


<class 'str'>


In [63]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1736
Prompt Tokens:693
Completion Tokens:1043
Total Cost:0.0007297499999999999


In [64]:
response

'The quiz you\'ve provided is a good starting point for assessing knowledge of machine learning concepts. However, there are several areas where improvements can be made to enhance clarity, engagement, and educational value. Here are some suggestions:\n\n### 1. **Question Variety and Depth**\n- **Increase Complexity**: Some questions are quite basic. Consider adding questions that require a deeper understanding or application of concepts.\n- **Include Real-World Scenarios**: Incorporate questions that present real-world scenarios where machine learning is applied, asking the quiz taker to identify the correct approach or method.\n\n### 2. **Answer Choices**\n- **Distractor Quality**: Ensure that distractor options (incorrect answers) are plausible and relevant. This will make the quiz more challenging and educational.\n- **More Options**: Consider providing more than four options for each question. This can increase the difficulty and require more critical thinking.\n\n### 3. **Clarity

In [53]:
quiz_response = quiz_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE
})

print(repr(quiz_response))


'{\n  "quiz": [\n    {\n      "question": "What does machine learning focus on?",\n      "options": {\n        "A": "Learning from data",\n        "B": "Writing code",\n        "C": "Hardware design",\n        "D": "Network security"\n      },\n      "answer": "A"\n    },\n    {\n      "question": "Which subfield of machine learning has shown significant advances recently?",\n      "options": {\n        "A": "Deep learning",\n        "B": "Reinforcement learning",\n        "C": "Supervised learning",\n        "D": "Unsupervised learning"\n      },\n      "answer": "A"\n    },\n    {\n      "question": "What is predictive analytics?",\n      "options": {\n        "A": "Applying ML to business problems",\n        "B": "Creating video games",\n        "C": "Designing websites",\n        "D": "Building hardware"\n      },\n      "answer": "A"\n    },\n    {\n      "question": "What do statistics and mathematical optimization provide for machine learning?",\n      "options": {\n        "A":

In [54]:
import json

quiz_data = json.loads(quiz_response)
print("✅ JSON parsed successfully")


✅ JSON parsed successfully


In [55]:
print(type(quiz_data))          # dict
print(quiz_data.keys())         # should contain 'quiz'
print(type(quiz_data["quiz"])) # list


<class 'dict'>
dict_keys(['quiz'])
<class 'list'>


In [52]:
print("TYPE:", type(response))
print("REPR:", repr(response))


TYPE: <class 'str'>
REPR: 'The quiz you\'ve provided is a good starting point for assessing knowledge of machine learning concepts. However, there are several areas where improvements can be made to enhance clarity, engagement, and educational value. Here are some suggestions:\n\n### 1. **Question Variety and Depth**\n- **Increase Complexity**: Some questions are quite basic. Consider adding questions that require a deeper understanding or application of concepts.\n- **Include Real-World Scenarios**: Incorporate questions that present real-world scenarios where machine learning is applied, asking the quiz taker to identify the correct approach or method.\n\n### 2. **Answer Choices**\n- **Distractor Quality**: Ensure that distractor options (incorrect answers) are plausible and relevant. This will make the quiz more challenging and educational.\n- **More Options**: Consider providing more than four options for each question. This can increase the difficulty and require more critical thi

In [57]:
quiz = quiz_data.get("quiz")


In [59]:

quiz_table_data = []

for q in quiz:
    mcq = q["question"]

    options = " | ".join(
        [f"{opt}: {text}" for opt, text in q["options"].items()]
    )

    correct = q["answer"]

    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct
    })


In [60]:
quiz_table_data

[{'MCQ': 'What does machine learning focus on?',
  'Choices': 'A: Learning from data | B: Writing code | C: Hardware design | D: Network security',
  'Correct': 'A'},
 {'MCQ': 'Which subfield of machine learning has shown significant advances recently?',
  'Choices': 'A: Deep learning | B: Reinforcement learning | C: Supervised learning | D: Unsupervised learning',
  'Correct': 'A'},
 {'MCQ': 'What is predictive analytics?',
  'Choices': 'A: Applying ML to business problems | B: Creating video games | C: Designing websites | D: Building hardware',
  'Correct': 'A'},
 {'MCQ': 'What do statistics and mathematical optimization provide for machine learning?',
  'Choices': 'A: Foundations | B: Applications | C: User interfaces | D: Data storage',
  'Correct': 'A'},
 {'MCQ': 'What does empirical risk minimization relate to?',
  'Choices': 'A: Machine learning algorithms | B: Data visualization | C: Software testing | D: Network protocols',
  'Correct': 'A'}]

In [61]:
quiz=pd.DataFrame(quiz_table_data)

In [62]:
quiz.to_csv("machinelearning.csv",index=False)

In [31]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_20_2025_21_44_18'